# About this Jupyter Notebook

@author: Yingding Wang

This notebook is originated from the example of kubeflow visual metric with KF Pipeline UI (https://github.com/kubeflow/pipelines/blob/master/samples/test/metrics_visualization_v1.py)

In [1]:
import sys
!{sys.executable} -m pip install --upgrade --user kfp==1.8.12

In [2]:
import kfp.dsl as dsl
import kfp
from kfp.components import create_component_from_func

# Advanced function
# Demonstrates imports, helper functions and multiple outputs
from typing import NamedTuple


@create_component_from_func
def confusion_visualization(matrix_uri: str = 'https://raw.githubusercontent.com/kubeflow/pipelines/master/samples/core/visualization/confusion_matrix.csv') -> NamedTuple('VisualizationOutput', [('mlpipeline_ui_metadata', 'UI_metadata')]):
    """Provide confusion matrix csv file to visualize as metrics."""
    import json

    metadata = {
        'outputs' : [{
          'type': 'confusion_matrix',
          'format': 'csv',
          'schema': [
            {'name': 'target', 'type': 'CATEGORY'},
            {'name': 'predicted', 'type': 'CATEGORY'},
            {'name': 'count', 'type': 'NUMBER'},
          ],
          'source': matrix_uri,
          'labels': ['rose', 'lily', 'iris'],
        }]
    }

    from collections import namedtuple
    visualization_output = namedtuple('VisualizationOutput', [
        'mlpipeline_ui_metadata'])
    return visualization_output(json.dumps(metadata))

@dsl.pipeline(
    name='confusion-matrix-pipeline',
    description='A sample pipeline to generate Confusion Matrix for UI visualization.'
)
def confusion_matrix_pipeline():
    confusion_visualization_task = confusion_visualization()
    # You can also upload samples/core/visualization/confusion_matrix.csv to Google Cloud Storage.
    # And call the component function with gcs path parameter like below:
    # confusion_visualization_task2 = confusion_visualization('gs://<bucket-name>/<path>/confusion_matrix.csv')

In [3]:
NAME_SPACE="kubeflow-kindfor"
EXPERIMENT_NAME="kf-metrics"

client = kfp.Client()
client.set_user_namespace(NAME_SPACE)

In [4]:
client.create_run_from_pipeline_func(pipeline_func=confusion_matrix_pipeline, arguments=None,
                                     experiment_name=EXPERIMENT_NAME, namespace=NAME_SPACE,
                                     pipeline_conf=dsl.PipelineConf(),
                                     mode = dsl.PipelineExecutionMode.V1_LEGACY
                                    )

RunPipelineResult(run_id=a6af6876-17d4-4573-a2f9-2662d1b921c9)